In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
import math
from tqdm import tqdm
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, BertTokenizer
from sklearn.model_selection import train_test_split
import gc
from tokenizers import ByteLevelBPETokenizer, processors

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)


if not torch.cuda.is_available():
    print("CUDA is not available on your system.")
else:
    # Print the number of CUDA devices
    print(f"Number of CUDA Devices: {torch.cuda.device_count()}\n")

    for i in range(torch.cuda.device_count()):
        print(f"Device {i}:")
        print(f"    Name: {torch.cuda.get_device_name(i)}")
        print(f"    Computational Capability: {torch.cuda.get_device_capability(i)}")
        print(f"    Total Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**2):.2f} MB\n")



cuda
Number of CUDA Devices: 1

Device 0:
    Name: NVIDIA GeForce RTX 3090
    Computational Capability: (8, 6)
    Total Memory: 24575.50 MB



In [4]:
data_path = 'datasets/tiny_stories/'

In [5]:
filename = 'tiny_stories.csv'
filepath = os.path.join(data_path, filename)

data = pd.read_csv(filepath)
data = data.dropna()

print(data.head())


   Unnamed: 0                                                 en  \
0           0  Once upon a time, there was a little boy named...   
1           1  Once upon a time, there was a normal boy named...   
2           2  Once upon a time, there was a fast car named S...   
3           3  Once there was a family with a very playful go...   
4           4  Once upon a time, there was a dog named Max. M...   

                                                  vi  
0  Ngày xửa ngày xưa, có một cậu bé tên Tim. Tim ...  
1  Ngày xửa ngày xưa, có một cậu bé bình thường t...  
2  Ngày xửa ngày xưa, có một chiếc xe tốc độ tên ...  
3  Có một gia đình có một con dê rất hay vui đùa....  
4  Ngày xửa ngày xưa, có một con chó tên là Max. ...  


In [6]:
N = len(data)
train_size = int(0.9 * N)
train_data = data[:train_size]
val_data = data[train_size:]

print(train_data.shape, val_data.shape)

(2339822, 3) (259981, 3)


In [7]:
from transformers import BertTokenizer, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Get the IDs
cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id
unk_id = tokenizer.unk_token_id
pad_id = tokenizer.pad_token_id

print("PAD Token ID:", pad_id)
print("UNK Token ID:", unk_id)
print("CLS Token ID:", cls_id)
print("SEP Token ID:", sep_id)

PAD Token ID: 1
UNK Token ID: 3
CLS Token ID: 0
SEP Token ID: 2


In [8]:
BLOCK_SIZE = 256
VOCAB_SIZE = tokenizer.vocab_size
print('Vocab size is ', VOCAB_SIZE)


Vocab size is  64000


In [9]:
def get_batch(data, block_size, batch_size):
    x = torch.zeros((batch_size, block_size), dtype=torch.long)
    y = torch.zeros((batch_size, block_size), dtype=torch.long)
    samples = data['vi'].sample(n=batch_size)

    for i, sample in enumerate(samples):
        summary_ids = tokenizer.encode(sample)
        summary_ids = [id for id in summary_ids if id != unk_id]
        if len(summary_ids) < block_size + 2:
            summary_ids = summary_ids + [pad_id] * (block_size + 2 - len(summary_ids))
        random_start = random.randint(0, len(summary_ids) - block_size - 2)
        x[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start:random_start + block_size], dtype=torch.long)
        y[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start + 1:random_start + block_size + 1], dtype=torch.long)

    return x, y

a, b = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=1)
print(a.shape, b.shape)
print(tokenizer.decode(a[0].tolist(), skip_special_tokens=False))
print(tokenizer.decode(b[0].tolist(), skip_special_tokens=False))

torch.Size([1, 256]) torch.Size([1, 256])
<s> Một ngày nọ, một cậu bé tên Tim tìm thấy một chiếc mai trong sân nhà mình. Cậu rất vui mừng. Tim muốn chơi với chiếc mai và nghĩ rằng "Mình có thể sáng tạo với cái mai này"Tim đến chỗ Sally, bạn của anh. Anh chỉ cho cô ấy thấy chiếc mai và Sally nói: "Wow! Chúng ta có thể làm gì với chiếc mai?" Tim nghĩ một lúc rồi nảy ra ý tưởng. Anh nói: 'Hãy đào một cái hố và làm một hồ bơi nhỏ để làm đồ chơi'Sally thích ý tưởng này. Họ bắt đầu đào bằng thuổng. Họ đào và đào. Chẳng mấy chốc, họ đã có một cái hố nhỏ. Tim đề nghị, "Hãy đặt nước vào trong lỗ" Họ đổ đầy nước vào hố. Bây giờ, họ có một bể bơi nhỏ để làm đồ chơi cho mình.Tim và Sally chơi với đồ chơi của họ trong hồ bơi. Họ đã rất vui vẻ. Họ hạnh phúc vì tìm thấy chiếc thuổng và có thể sáng tạo với nó. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Một ngày nọ, mộ

In [ ]:
del model, xb, yb  # delete the tensor variable
torch.cuda.empty_cache()  # clear unused memory in PyTorch
gc.collect()  # call Python garbage collector

In [15]:
N_EMB = 800
N_LAYERS = 4
N_HEADS = 4
DROPOUT = 0.2


def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones(sz, sz)) == 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def get_sine_position_encodings(length, dim):
    pos = torch.arange(length, dtype=torch.float32).reshape(-1, 1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pos_encodings = torch.zeros(length, dim)
    pos_encodings[:, 0::2] = torch.sin(pos * div_term)
    pos_encodings[:, 1::2] = torch.cos(pos * div_term)
    return pos_encodings

class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb, block_size, n_layers, n_heads, dropout=0.2):
        super(LanguageModel, self).__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.block_size = block_size

        encoder_layer = nn.TransformerEncoderLayer(d_model=n_emb, nhead=n_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.feed_forward = nn.Sequential(
            nn.Linear(n_emb, 4 * n_emb),
            nn.ReLU(),
            nn.Linear(4 * n_emb, n_emb)
        )

        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device))

        x = token_emb + position_emb
        x_transform = x.clone()
        mask = generate_square_subsequent_mask(T).to(device)

        x_transform = self.transformer_encoder(x_transform.permute(1, 0, 2), mask=mask)
        x_transform = x_transform.permute(1, 0, 2)
        x = x + x_transform

        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None


    def generate(self, idx, max_new_tokens, temperature=1.0, stop_token=False):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self.forward(idx_cond)

            # Scale logits by the temperature
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
            if stop_token and idx_new.item() == sep_id:
                break
        return idx

# Create model, optimizer
model = LanguageModel(vocab_size=VOCAB_SIZE, block_size=BLOCK_SIZE, n_emb=N_EMB, n_layers=N_LAYERS, \
    n_heads=N_HEADS, dropout=DROPOUT).to(device)

print(f'Number of parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

Number of parameters 131176992


In [14]:
model_path = os.path.join(data_path, "vietnamese_tiny_stories_4layers.pth")
print(model_path)
model = torch.load(model_path)

datasets/tiny_stories/vietnamese_tiny_stories_4layers.pth


In [16]:
EARLY_STOP = 50
N_EPOCHS = 20000
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

last_val_loss = 1e9
early_stop = EARLY_STOP

for steps in range(N_EPOCHS):
    model.train()
    xb, yb = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Delete xb, yb and free GPU memory
    del xb, yb
    torch.cuda.empty_cache()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(model, val_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
        print('Validation loss:', val_loss)
        # if val_loss >= last_val_loss:
        #     early_stop -= 1
        #     if early_stop == 0:
        #         print('Early stop!')
        #         break
        # else:
        #     early_stop = EARLY_STOP
        #     last_val_loss = val_loss

Step: 0 Training Loss: 11.12340259552002
Validation loss: 9.132868766784668
Step: 100 Training Loss: 3.631944417953491
Validation loss: 3.5089168548583984
Step: 200 Training Loss: 2.831491470336914
Validation loss: 3.315843105316162
Step: 300 Training Loss: 2.9097506999969482
Validation loss: 2.712236166000366
Step: 400 Training Loss: 2.764423370361328
Validation loss: 2.630145788192749
Step: 500 Training Loss: 2.649531602859497
Validation loss: 2.380825996398926
Step: 600 Training Loss: 2.355130910873413
Validation loss: 2.4677348136901855
Step: 700 Training Loss: 2.51491641998291
Validation loss: 2.37542462348938
Step: 800 Training Loss: 2.2452855110168457
Validation loss: 2.074643611907959
Step: 900 Training Loss: 2.143148899078369
Validation loss: 2.309785842895508
Step: 1000 Training Loss: 2.4260692596435547
Validation loss: 2.0690059661865234
Step: 1100 Training Loss: 2.2619242668151855
Validation loss: 2.1847000122070312
Step: 1200 Training Loss: 2.1206932067871094
Validation lo

In [17]:
model_path = os.path.join(data_path, "vietnamese_tiny_stories_4layers.pth")
torch.save(model, model_path)

In [21]:
starting_tokens = 'Hôm nay trời nắng. Mèo con đi học chẳng mang thứ gì. '
encoded_start = tokenizer.encode(starting_tokens)
encoded_start.pop(-1)
len_starting_tokens = len(encoded_start)

idx = torch.tensor(encoded_start).reshape(1, len_starting_tokens).to(device)
model.eval()
N_SAMPLES = 10
for _ in range(N_SAMPLES):
    generation = model.generate(idx, max_new_tokens=2000, temperature=0.5, stop_token=True)[0].tolist()
    story = tokenizer.decode(generation, skip_special_tokens=True)

    print('Story ', _ + 1, ':')
    print(story)
    print('\n')






Story  1 :
Hôm nay trời nắng. Mèo con đi học chẳng mang thứ gì. Nó nhìn thấy một cái cây to, một cái ao xinh xắn. Mèo con rất vui vẻ. Nó muốn chơi với cái ao đó.Mèo con chơi với cái ao cả ngày. Nó lăn nó quanh hồ, giả vờ là một hoàng tử. Con mèo con rất vui vẻ. Khi đến lúc về nhà, nó chào tạm biệt cái ao và đi ngủ. Nó biết mình sẽ sớm được chơi với cái ao xinh đẹp này thôi.


Story  2 :
Hôm nay trời nắng. Mèo con đi học chẳng mang thứ gì. Nó thấy một cái cây lớn với quả ngon lành trên đó. Mèo con rất vui sướng. Nó muốn ăn trái cây đó.Mèo con bắt đầu ăn trái cây. Nó không biết rằng trái cây này rất ngon. Nó cố gắng lấy trái cây nhưng nó quá cao nên không với tới được. Nó cảm thấy buồn bã.Rồi một cô gái tốt bụng nhìn thấy con mèo. Cô muốn giúp đỡ nó. Cô trèo lên cây và hái trái cây. Mèo con rất vui vẻ. Mèo con và cô gái chơi với nhau suốt ngày.


Story  3 :
Hôm nay trời nắng. Mèo con đi học chẳng mang thứ gì. Nó thấy bạn bè đang chơi trò chơi. Mèo con đang vui vẻ.Con mèo con hỏi bạn mình